In [14]:
!pip install langgraph

In [16]:
from typing import TypedDict
from langgraph.graph import StateGraph # Corrected import
from langgraph.graph.message import add_messages

class ResumeState(TypedDict):
    resume_text: str
    skills: list[str]
    score: int
    decision: str

def parse_resume(state: ResumeState) -> dict:
    print("--- Executing Node: parse_resume ---")
    resume = state["resume_text"]
    skills = []
    if "python" in resume.lower():
        skills.append("Python")
    if "java" in resume.lower():
        skills.append("Java")
    if "management" in resume.lower():
        skills.append("Management")
    return {"skills": skills}

def score_candidate(state: ResumeState) -> dict:
    print("--- Executing Node: score_candidate ---")
    score = len(state.get("skills", [])) * 20
    return {"score": score}

def make_decision(state: ResumeState) -> dict:
    print("--- Executing Node: make_decision ---")
    score = state.get("score", 0)
    if score >= 60:
        decision = "Interview"
    elif score >= 40:
        decision = "Second Review"
    else:
        decision = "Reject"
    return {"decision": decision}

# Changed from Graph to StateGraph
graph = StateGraph(ResumeState) # Changed from Graph to StateGraph
graph.add_node("parse", parse_resume)
graph.add_node("score", score_candidate)
graph.add_node("decide", make_decision)

graph.set_entry_point("parse")
graph.add_edge("parse", "score")
graph.add_edge("score", "decide")
graph.set_finish_point("decide")

app = graph.compile()

initial_state = {
    "resume_text": "Experienced Python developer with Java and management experience"
}
final_state = app.invoke(initial_state)

print("\n" + "="*50)
print("FUNCTIONAL API - FINAL STATE:")
print(final_state)
print("="*50)

--- Executing Node: parse_resume ---
--- Executing Node: score_candidate ---
--- Executing Node: make_decision ---

FUNCTIONAL API - FINAL STATE:
{'resume_text': 'Experienced Python developer with Java and management experience', 'skills': ['Python', 'Java', 'Management'], 'score': 60, 'decision': 'Interview'}
